# Open Class Rooms - Formation Ingénieur Machine Learning
## Projet : Définissez votre stratégie d'apprentissage

Created by: Thomas Durand-Texte, Décembre 2022

# Import des packages et chargement des données
Les données correspondent à un temps estimatif de travail des différents projets

In [2]:
import pandas as pd
import numpy as np

data = pd.read_csv('projets.csv') ;
display( data )

,projet,durée,durée cours,durée supplémentaire
0,Application santé,70h,25h,30h
1,Besoin conso batiements,100h,47h,40h
2,Segmentez clients e-commerce,70h,25h,20h
3,Catégorisez questions,80h,8h,80h
4,Classez images algo Deep Learning,100h,40h,60h
5,Dévelopez une preuve de concept,30h,1h,20h
6,Compétition Kaggle,50h,0h,50h


# Vérification du type des colones et conversion des temps/durée str->int
* Toutes les colonnes sont vérifiées avec une boucle for
* si la colonne est déjà en int, ou si 'durée' n'est pas présent dans la key, alors la colonne est inchangée

In [4]:
for key in data.keys():
    if (data[key].dtype == 'int64') or ( not 'durée' in key): continue ;
    data[key] = pd.to_numeric(data[key].str.replace('h',''), errors='coerce' ) ;
# for #
print('dtypes:\n', data.dtypes)

dtypes:
 projet                  object
durée                    int64
durée cours              int64
durée supplémentaire     int64
dtype: object


# Calcul du temps total pour chaque projet

In [28]:
data['durée totale'] = data['durée'] + data['durée cours'] + data['durée supplémentaire'] ;
display( data )        

,projet,durée,durée cours,durée supplémentaire,durée totale
0,Application santé,70,25,30,125
1,Besoin conso batiements,100,47,40,187
2,Segmentez clients e-commerce,70,25,20,115
3,Catégorisez questions,80,8,80,168
4,Classez images algo Deep Learning,100,40,60,200
5,Dévelopez une preuve de concept,30,1,20,51
6,Compétition Kaggle,50,0,50,100


# Temps initial estimatif en heures
1. calcul du nombre de jours de janvier à juin inclu
1. calcul du nombre de semaines (entières)
1. calcul du nombre d'heures à disposition sur une base de 35h/semaine
1. somme des heures des différents projets (estimation) et comparaison avec le nombre d'heures à disposition

In [29]:
n_jours = np.sum( [31,28,31,30,31,30]) ;
n_sem = n_jours // 7 +2 ;  # floor + ajout 2 semaines
n_jours_restant = n_jours - 7*n_sem ;
n_heures_dispo = 35 * n_sem ;
heures_total_estim = data['durée totale'].sum() ;
print( 'nombre de jours initial : {:}'.format(n_jours) ) ;
print( 'nombre de semaines : {:}'.format(n_sem) ) ;
print( 'nombre de jours restants : {:}'.format(n_jours_restant) ) ;
print( "nombre d'heures disponibles :{:}".format(n_heures_dispo) ) ;
print( "nombre d'heures total estimaté : {:}".format(heures_total_estim) ) ;

nombre de jours initial : 181
nombre de semaines : 27
nombre de jours restants : -8
nombre d'heures disponibles :945
nombre d'heures total estimaté : 946


# Estimation de la durée de chaque projet
Valeurs en semaine, sur une base de 35h/semaine, arrondi à l'entier supérieur

In [30]:
key = 'durée estimée (semaines)' ;
data[key] = ( data['durée totale'] / 35 + 1. ).astype('int') ;
print('nombre de semaines initial (janvier à juin) : {:}'.format( n_sem ) ) ;
print('nombre de semaines estimé : {:}'.format( data[key].sum() ) ) ;
display( data )

nombre de semaines initial (janvier à juin) : 27
nombre de semaines estimé : 30


,projet,durée,durée cours,durée supplémentaire,durée totale,durée estimée (semaines)
0,Application santé,70,25,30,125,4
1,Besoin conso batiements,100,47,40,187,6
2,Segmentez clients e-commerce,70,25,20,115,4
3,Catégorisez questions,80,8,80,168,5
4,Classez images algo Deep Learning,100,40,60,200,6
5,Dévelopez une preuve de concept,30,1,20,51,2
6,Compétition Kaggle,50,0,50,100,3


# Estimation des dates de soutenance
1. Conversion des durées cumulées en *timedelta*
1. Ajout à une date initiale le 02/01/2023

In [32]:
duree_cumsum = 7*data['durée estimée (semaines)'].cumsum() ;
data['date soutenance (estimation)'] = \
    pd.to_datetime( '02/01/2023',format='%d/%m/%Y') \
    + pd.to_timedelta( duree_cumsum.astype(str) + ' days') ;
display(data)

,projet,durée,durée cours,durée supplémentaire,durée totale,durée estimée (semaines),date soutenance (estimation)
0,Application santé,70,25,30,125,4,2023-01-30
1,Besoin conso batiements,100,47,40,187,6,2023-03-13
2,Segmentez clients e-commerce,70,25,20,115,4,2023-04-10
3,Catégorisez questions,80,8,80,168,5,2023-05-15
4,Classez images algo Deep Learning,100,40,60,200,6,2023-06-26
5,Dévelopez une preuve de concept,30,1,20,51,2,2023-07-10
6,Compétition Kaggle,50,0,50,100,3,2023-07-31


# Création de la chaine de caractère à copier/coller dans la fiche de début de formation

In [33]:
str_out = '' ;
for i, date in enumerate( data['date soutenance (estimation)'].tolist() ):
    date_str = date.strftime('%d/%m/%Y') ; 
    str_out += '\n-Projet {:}: {:}'.format( i+1, date_str ) ;
# for i, date #
print(str_out) ;


-Projet 1: 30/01/2023
-Projet 2: 13/03/2023
-Projet 3: 10/04/2023
-Projet 4: 15/05/2023
-Projet 5: 26/06/2023
-Projet 6: 10/07/2023
-Projet 7: 31/07/2023
